In [1]:
!cp /content/drive/MyDrive/pagila_schema.sql .

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline,AutoModelForCausalLM

In [3]:
import re

with open("pagila_schema.sql", "r") as f:
    schema_full = f.read()

table_defs = re.findall(r'CREATE TABLE .*?;\n', schema_full, re.DOTALL)

schema_clean = "\n".join(table_defs)

print(schema_clean[:2000])

CREATE TABLE public.customer (
    customer_id integer DEFAULT nextval('public.customer_customer_id_seq'::regclass) NOT NULL,
    store_id integer NOT NULL,
    first_name text NOT NULL,
    last_name text NOT NULL,
    email text,
    address_id integer NOT NULL,
    activebool boolean DEFAULT true NOT NULL,
    create_date date DEFAULT CURRENT_DATE NOT NULL,
    last_update timestamp with time zone DEFAULT now(),
    active integer
);

CREATE TABLE public.actor (
    actor_id integer DEFAULT nextval('public.actor_actor_id_seq'::regclass) NOT NULL,
    first_name text NOT NULL,
    last_name text NOT NULL,
    last_update timestamp with time zone DEFAULT now() NOT NULL
);

CREATE TABLE public.category (
    category_id integer DEFAULT nextval('public.category_category_id_seq'::regclass) NOT NULL,
    name text NOT NULL,
    last_update timestamp with time zone DEFAULT now() NOT NULL
);

CREATE TABLE public.film (
    film_id integer DEFAULT nextval('public.film_film_id_seq'::regclass)

In [4]:
question = "which films in the Action category have generated the highest total rental revenue?"

In [5]:
prompt = f"""### Postgres SQL tables, with their properties:
{schema_clean}

### Task
{question}

### SQL query
"""

In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_id = "defog/sqlcoder-7b-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

sql_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
output = sql_generator(prompt, max_new_tokens=256, do_sample=False)[0]["generated_text"]

generated_sql = output.split("### SQL query")[-1].strip()

print("✅ SQL Query:")
print(generated_sql)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ SQL Query:
SELECT f.title, SUM(r.amount) AS total_rental_revenue FROM public.film f JOIN public.film_category fc ON f.film_id = fc.film_id JOIN public.rental r ON f.film_id = r.inventory_id WHERE fc.category_id = (SELECT category_id FROM public.category WHERE name = 'Action') GROUP BY f.title ORDER BY total_rental_revenue DESC NULLS LAST;
